### 1. Расчет Lifetime

#### Постановка задачи:

Для пользователей из датасета data.tsv необходимо посчитать метрику lifetime в днях (среднее время жизни) с точки зрения покупок

In [1]:
import pandas as pd
import datetime
from datetime import datetime,timedelta

# загрузим исходный датасет data.tsv
data = pd.read_csv('data_lifetime.tsv', sep='\t')
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8


In [2]:
def convert_to_datetime(row):
    """
    Функция преобразует данные типа date в тип datetime
    """
    return datetime.strptime(row['date'], '%d.%m.%Y %H:%M')

# Создадим столбец datetime с даннами типа datetime с помощью функции convert_to_datetime
data['datetime'] = data.apply(convert_to_datetime, axis=1)

In [3]:
import time
def make_unix_time(row):
    """
    Функция преобразует данные типа datetime в unix-формат (количество секунд, прошедшее с 1 января 1970 года)
    """
    return time.mktime(row['datetime'].timetuple())

# Создадим столбец с данными формата unixtime с помощью функции make_unix_time
data['unixtime'] = data.apply(make_unix_time, axis=1)
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime,unixtime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00,1.475699e+09
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00,1.476038e+09
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00,1.475627e+09
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00,1.475699e+09
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00,1.475287e+09


In [4]:
# Сгруппируем датафрейм data по столбцу user_id, посчитав для столбца unixtime макс. и миним. значение для каждого пользователя.
data_min_max = data.groupby('user_id').agg(['min','max']).reset_index()[['user_id','unixtime']]

# Создадим новый столбец diff
data_min_max['diff'] = data_min_max['unixtime']['max']-data_min_max['unixtime']['min']
data_min_max.head()

user_id      unixtime                    diff
                    min           max          
0    1010  1.475699e+09  1.475699e+09       0.0
1    1036  1.476038e+09  1.476038e+09       0.0
2    1041  1.475627e+09  1.475699e+09   71760.0
3    1042  1.475287e+09  1.475934e+09  647340.0
4    1047  1.475318e+09  1.475693e+09  374520.0

In [5]:
# Исключим из расчета пользователей, у которых разница diff равна 0
data_min_max = data_min_max[data_min_max['diff']!=0]
data_min_max.head()

user_id      unixtime                    diff
                    min           max          
2    1041  1.475627e+09  1.475699e+09   71760.0
3    1042  1.475287e+09  1.475934e+09  647340.0
4    1047  1.475318e+09  1.475693e+09  374520.0
5    1052  1.475824e+09  1.475945e+09  121020.0
6    1057  1.475569e+09  1.475970e+09  401700.0

In [6]:
# Посчитаем среднее значение столбца 'diff'. Это и будет значение Lifetime (в секундах)
lifetime = data_min_max['diff'].mean()
print('Значение lifetime равно {:.1f} c.'.format(lifetime))

Значение lifetime равно 301143.2 c.


In [7]:
# Переведем это значение в дни. (1 день= 86400 с). 
lifetime_day= lifetime/86400
# Ответ на вопрос задачи 
print('Значение lifetime равно {:.1f} дн.'.format(lifetime_day)) 

Значение lifetime равно 3.5 дн.


### 2. Расчет показателя ROI (return on investment)

#### Постановка задачи:

Для статистики покупок в файле data_roi_.txt необходимо посчитать ROI в разрезе типов источников трафика

##### Имеется ряд условий:

1. Комиссия по типу канала:

    - cpa-partners (burgerclub 30%, food-delivery 25%)

    - cpc-partners (city-magazine 7 рублей, foody 9 рублей)

    - остальные каналы (фиксированная комиссия 4 рубля)
    

2. Типы определяются следующим образом:

    - если source равен 'google' или 'yandex', то проверяем medium:
    
        - для medium 'seo' или 'sem' тип источника равен 'search engines seo'
        
        - для medium 'brand' - тип источника равен 'search engines brand'
        
        - для остальных случаев тип источника равен 'search engines undefined'
        
    - если условие не выполнено, то тип источника равен 'other'


In [8]:
#Создадим словари, в которых будут содержаться комиссии трех типов

#cpa-partners (burgerclub 30%, food-delivery 25%)
cpa_commission = {
    'burgerclub': 0.3,
    'food-delivery': 0.25 }

# cpc-partners (city-magazine 7 рублей, foody 9 рублей)
cpc_commission = {
    'city-magazine': 7,
    'foody': 9}
# остальные каналы (фиксированная комиссия 4 рубля)
fixed_commission = 4  

In [9]:
def costs_classification( amount_paid, source ):
    """
    Функция по названию канала source возвращает размер комиссии.
    """
    # для cpa-partners возвращаем долю от выручки
    if source in cpa_commission:
        return amount_paid * cpa_commission[ source ] 
   
    # для cpc-partners стоимость расходов постоянная
    if source in cpc_commission:
        return cpc_commission[ source ]
    
    # для остальных каналов возвращаем фиксированную стоимость fixed_commission = 4руб
    return fixed_commission

In [10]:
def source_type_classification(line):
    """
    Функция классифицирует источник трафика в зависимости от значения source и medium.
    """
    source = line[5]
    medium = line[4]
    if source == 'google' or source == 'yandex':
        if medium == 'seo' or medium == 'sem':
            source = 'search engines seo'
        elif medium == 'brand':
            source = 'search engines brand'
        else:
            # если вдруг встретится другой вариант, то ставим "undefined"
            source = 'search engines undefined'
    else:
        source = 'other'
    return source

In [11]:
def expense_and_income( line ):
    """
    Функция для строки line возвращает тип источника трафика, итоговый расход и доход
    """
    source = source_type_classification(line) # тип источника трафика
    amount_paid = float( line[-1].replace( ',', '.' ) ) # доход
    cost = float( line[6].replace( ',', '.' ) ) # расход
    partner_comission = costs_classification( amount_paid, source ) # комиссия партнера
    return source, cost+partner_comission, amount_paid

In [12]:
# основной блок
# инициализация словаря, в котором будет содержаться доходы и расходы по типу источников трафика
roi_stats = {} 

Started = True # метка  первой строки с заголовками
# построчное чтение исходного файла data_roi.txt
with open( 'data_roi_.txt', 'r' ) as f:
    for line in f:
        if Started:
            Started = False # не учитываем первую строку с заголовками
        else:
            line = line.strip().split('\t')
            source, cost, income = expense_and_income( line )
            if source in roi_stats:
                roi_stats[source]['cost'] += cost
                roi_stats[source]['income'] += income
            else:
                roi_stats[source] = {}
                roi_stats[source]['cost'] = cost
                roi_stats[source]['income'] = income

for source, data in roi_stats.items():
    # в словарь data добавим ключ ROI,значение которого расчитывается как отношение прибыли к расходам 
    data['roi'] = (data['income'] - data['cost']) / data['cost']
    
    # Ответ на вопрос задачи - вывод в цикле ROI для каждого типа источника трафика 
    print('ROI  {} = {:.2f}'.format(source, data['roi']))

ROI  search engines seo = 3.24
ROI  other = 3.05
ROI  search engines brand = 3.51


### 3. Расчет рейтинга фильмов

#### Постановка задачи:

По данным из двух датасетов с фильмами и рейтингами (movies.csv, ratings.csv) необходимо найти год выпуска фильмов с максимальным средним рейтингом.

In [13]:
import pandas as pd
import numpy as np

# загрузим исходные датасеты ratings.csv и movies.csv
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [14]:
# объединим два датасета в один по полю movieId
join = ratings.merge(movies, on='movieId', how='left')
join.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [15]:
# создадим список годов с 1950 по 2010
years = range(1950,2011)

# создадим функцию production_year, которая каждой строке из названия фильма возращает год выпуска
def production_year(row):
    """Возвращает год выпуска фильма по названию фильма"""
    
    for y in years:
        if str(y) in row['title']:
            return y
    # Если ни один год не найден, возвращаем 1900
    return 1900   

In [16]:
# добавим новый столбец 'year', используя функцию production_year
join['year'] =join.apply(production_year,axis =1)
join.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [17]:
# посчитаем средний рейтинг всех фильмов для каждого значения столбца 'year'
# отсортируем результат по убыванию рейтинга
join.groupby('year').mean().sort_values('rating',ascending = False).head()

# Ответ на вопрос задач: в 1957 году были выпущены фильмы, которые имеют максимальный средний рейтинг пользователей

,userId,movieId,rating,timestamp
year,,,,
1957,360.933544,2750.965190,4.014241,1.083707e+09
1972,359.694878,3983.538976,4.011136,1.122759e+09
1952,346.394737,4107.796053,4.000000,1.090512e+09
1954,358.228324,2867.661850,3.994220,1.070591e+09
1951,347.106996,2605.588477,3.983539,1.052714e+09
